In [9]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd

## MADRID

#### Coordenas de nuestras compañias

In [60]:
gdf_coodenadas_Madrid = pd.read_csv('../output/Madrid_coordenadas_clean.csv')
gdf_coodenadas_Madrid_fin = gdf_coodenadas_Madrid.drop(gdf_coodenadas_Madrid.columns[[4]], axis = 'columns').copy()
gdf_coodenadas_Madrid_fin.loc[:,'category'] = ['company', 'company', 'company', 'company']

gdf_coodenadas_Madrid_fin.rename(columns={'latitude': 'latitud', 'longitude': 'longitud'}, inplace=True)
gdf_coodenadas_Madrid_fin

,Unnamed: 0,name,latitud,longitud,category
0,0,Digital Assets Deployment,40.420460,-3.678121,company
1,1,Daily Flat Rental,40.412323,-3.703248,company
2,3,ticketea,40.445516,-3.706176,company
3,4,BuyVIP,40.416741,-3.703250,company


#### Coordenadas de nuestras condiciones

In [91]:
## BARES
closest_bars_to_my_companies = pd.read_csv('../output/closest_bars_to_my_companies.csv')
closest_bars_to_my_companies

,Unnamed: 0,name,latitud,longitud,category
0,0,La Castela,40.420435,-3.676564,bars
1,1,Madrid cerrajeros,40.416553,-3.703803,bars
2,2,REEDIFIC,40.446784,-3.703803,bars
3,3,Madrid cerrajeros,40.416553,-3.703803,bars


In [67]:
# CONFERENCES
closest_conferences_to_my_companies = pd.read_csv('../output/coordenadas_conferences.csv')
closest_conferences_to_my_companies

,Unnamed: 0,name,latitud,longitud,category
0,0,Elzalburu,40.436095,-3.691326,conference
1,1,hokena rung,40.411633,-3.707981,conference
2,2,Espacio Efímeral,40.453630,-3.704104,conference
3,3,hokena rung,40.411633,-3.707981,conference


In [22]:
# SCHOOLS
closest_school_to_my_companies = pd.read_csv('../output/coordenadas_schools.csv')
closest_school_to_my_companies

,Unnamed: 0,name,latitud,longitud,category
0,0,Nuestra Señora De La Almudena,40.419158,-3.677935,school
1,1,CEIP Isabel la Catolica,40.426336,-3.699751,school
2,2,Centro educativo Maria Inmaculada Fuencarral,40.427867,-3.703431,school
3,3,CEIP Isabel la Catolica,40.426336,-3.699751,school


#### DF FINAL PARA PODER PINTAR NUESTRO MAPA

In [92]:
df_condiciones = pd.concat([gdf_coodenadas_Madrid_fin, closest_bars_to_my_companies, closest_conferences_to_my_companies, closest_school_to_my_companies], axis=0)
df_condiciones

,Unnamed: 0,name,latitud,longitud,category
0,0,Digital Assets Deployment,40.420460,-3.678121,company
1,1,Daily Flat Rental,40.412323,-3.703248,company
2,3,ticketea,40.445516,-3.706176,company
3,4,BuyVIP,40.416741,-3.703250,company
0,0,La Castela,40.420435,-3.676564,bars
1,1,Madrid cerrajeros,40.416553,-3.703803,bars
2,2,REEDIFIC,40.446784,-3.703803,bars
3,3,Madrid cerrajeros,40.416553,-3.703803,bars
0,0,Elzalburu,40.436095,-3.691326,conference
1,1,hokena rung,40.411633,-3.707981,conference


In [94]:
map_madrid = Map(location= [40.42955,  -3.6793], zoom_start = 15)


In [97]:
for i,row in df_condiciones.iterrows():
    distrito = {
        "location" : [row["latitud"], row["longitud"]],
        "tooltip" : row["name"]
    }
    
    if row["category"] == "bars":
        icon = Icon( color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "white"
        )
    elif row["category"] == "conference":
        icon = Icon( color = "blue",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "white"
        )
    elif row["category"] == "school":
        icon = Icon( color = "orange",
                    prefix = "fa",
                    icon = "fa-graduation-cap",
                    icon_color = "white"
        )
    else:
        icon = Icon( color = "red",
                    prefix = "fa",
                    icon = "fa-building-o",
                    icon_color = "black" )
    Marker (**distrito,icon = icon).add_to(map_madrid)
map_madrid

In [96]:
map_madrid.save('../input/mapa_madrid_condiciones')

### CALCULO DE LAS DISTANCIAS ENTRE NUESTRAS COORDENADAS

#### En este momento ya hemos decidido entre dos diferentes ubicaciones:
* Al lado de la empresa: Digital Assets Deployment
* o de la empresa: Daily Flat Rental

#### Estas dos empresas son las que más encajan con nuestras condiciones pero para ser un buen analista vamos a sacar distacias que hay entre:
* La empresa Digital Assets Deployment y el colegio Nuesra Señora de la Almudena y el bar La castela
* Y la empresa Daily Flat rental con el bar El Rincón del Cabo y la conferencia hokena rung

#### De esta manera nuestras estadísticas serán más aproximadas y por tanto nuestro margen de error será menor

In [71]:
import math

In [72]:
def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [77]:
Digital_Assets_Deployment_coord = 40.420460, -3.678121
condiciones = {
    'La Castela': (40.420435, -3.676564),
    'Nuestra Señora De La Almudena': (40.419158, -3.677935)

}


for cond, coord in condiciones.items():
    distance = haversine(Digital_Assets_Deployment_coord, coord)
    print(cond, distance)

La Castela 131.87201413963822
Nuestra Señora De La Almudena 145.67066642632273


In [78]:
Daily_Flat_Rental = 40.412323, -3.703248
condiciones = {
    'El Rincón del Cabo': (40.411716, -3.703075),
    'hokena rung': (40.411633, -3.707981)
}

for cond, coord in condiciones.items():
    distance = haversine(Digital_Assets_Deployment_coord, coord)
    print(cond, distance)

El Rincón del Cabo 2326.240283198325
hokena rung 2712.54532542554


#### CONCLUSIONES
* Como podemos observar, las distancias de la empresa Digital Assets Deployment con nuestras conciones son mucho menores y por ello nos quedaremos ahi 
* Por tanto nuestra ubicacion en el mapa de Madrid de nuestra empresa tendrá las siguientes coordenadas:
    - Latitud: 40.420460 
    - Longitud: -3.678121

In [82]:
map_1 = folium.Map(location = [40.420460, -3.678121], zoom_start = 15)

In [83]:
Mi_empresa = Marker(location=[40.420460, -3.678121], tooltip = "Mi empresa")
Mi_empresa.add_to(map_1)
map_1

In [93]:
map_1.save('../input/mapa_opcion_madrid')